# Avito Demand Prediction Challenge
Nesta competição, o Avito - site de anúncios da Rússia, similar ao Mercado Livre no Brasil - está disposto a descobrir qual a relação entre a descrição completa de um anúncio, seu contexto e demanda histórica anúncios semelhantes e o sucesso (ou fracasso) de vendas daquele produto. Para isto, disponibilizou datasets com todas essas informações e qual a probabilidade daqueles produtos venderem.

Para isto, iremos utilizar métodos de mineração de dados para predizer a probabilidade de um certo conjunto de produtos vender, usando seu anúncio e informações relevantes correspondentes. Inicialmente, utilizamos apenas um dataset disponível com muitas informações que apararentemente são relevantes para treinar nossos métodos. Fizemos uso dos seguintes métodos: SVM, Árvore de Decisão e Gradient Boosting. Em cada um deles, treinamos com o mesmo dataset e calculamos seu score com o mesmo datatset de teste. Os resultados são mostrados ao decorrer do relatório.


In [8]:
#importing important modules
import numpy as np # linear algebra
from pandas import read_csv, SparseSeries # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer, CountVectorizer
from sklearn.model_selection import KFold #KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import svm #SVM
from sklearn import tree # DecisionTreeClassifier
from nltk.corpus import stopwords
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
# Any results you write to the current directory are saved as output.

## Preparação dos dados



In [2]:
#reading the csv's
train = read_csv('../input/train.csv')
test = read_csv('../input/test.csv')
print ('Data loaded')

Data loaded


Abaixo, mostramos o que há em cada um dos datasets que lemos. O objeto *train* possui os dados que serão usados no treino. Nele, também estão as probabilidades correspondentes a cada anúncio. O objeto *test* será usado para o teste. No *test*, diferente do *train*, não há esses valores de probabilidade, que deverão ser "descobertos" pelo método que avaliarmos o melhor.

In [3]:
print('Train Columns: ',train.columns)
print('Test Columns: ',test.columns)

Como não serão usadas todas as informações disponíveis no dataset, estamos retirando as suas colunas correspondentes a essas informações que não serão utilizadas.

In [3]:
#dropping some columns
train = train.drop(['image', 'image_top_1'], axis = 1)
test = test.drop(['image', 'image_top_1'], axis = 1)

Y_df = train['deal_probability']
train_df = train.drop(['item_id','activation_date','user_type', 'title','description','deal_probability'], axis = 1)
enc_df = train.append(test).drop(['item_id','activation_date','user_type', 'title','description','deal_probability'], axis = 1)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


Uma visualização melhor dos dados pode mostrar quais colunas são realmente importantes para o target. Logo de cara, *user_type* ou *user_type* não apresentam, intuitivamente, influência sobre a probabilidade. A Descrição e o título, por sua vez, tratam-se de textos e configuram um cenário que há de ser tratado distintamente dos demais dados categóricos.

Um bom teste seria realizar um treino base com as colunas base.

In [4]:
train

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,0.20797
5,51e0962387f7,bbfad0b1ad0a,Татарстан,Чистополь,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300.0,9,2017-03-28,Private,0.80323
6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125,2017-03-23,Private,0.00000
7,6b71309d6a8a,fef86baa002c,Пермский край,Пермь,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500.0,61,2017-03-25,Private,0.80323
8,c5b969cb63a2,055825270190,Оренбургская область,Оренбург,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500.0,85,2017-03-17,Private,0.00000
9,b1570962e68c,f9e8f831d94c,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400.0,136,2017-03-22,Company,0.00000


Abaixo, todos os dados que serão usados para treino estão sendo codificados para valores entre 0 e número de classes menos 1.

In [6]:
#transforming columns
enc = {}
for col in train_df.columns:
    enc[col] = LabelEncoder()
    enc[col].fit(enc_df[col].values.astype(str))
    train_df[col] = enc[col].fit_transform(train_df[col].values.astype(str))
y_enc = LabelEncoder()
Y_df = y_enc.fit_transform(Y_df)

print ("Label Encoding done")

Label Encoding done


## Explicar o que foi feito


In [11]:
train_size = 100000
desc_df = train['description'][:train_size]
print ('Starting vectorizing')
hash_vect = HashingVectorizer(n_features = 2**10, encoding = 'KOI8-R', stop_words = stopwords.words('russian'))
count_vect = CountVectorizer(encoding = 'KOI8-R', stop_words = stopwords.words('russian'))

desc_vec = hash_vect.fit_transform(desc_df.values.astype(str))
sparse_desc = SparseSeries(desc_vec.toarray().ravel(), fill_value=0)
print ("Vectorizing done")

Starting vectorizing
Vectorizing done


## Iniciando o treinamento com o dataset
Nesta parte, será iniciado o treinamento com o dataset que foi pré-processado acima. Iremos utilizar os métodos: DecisionTreeClassifier, SVM e GradientBoosting, todos eles presentes no scikit-learn.

Como o dataset é absurdamente grande, ele será dividido em pequenas partes para ser testado.  Algumas partes serão usadas apenas para o fit e outras serão usadas para testar o quanto o método está acertando. Ao final, utilizaremos a média dos acertos para verificar qual método foi mais eficiente.

In [12]:
train_df_fit = train_df[:train_size]
train_df_fit['description'] = sparse_desc[:train_size]
Y_df_fit = Y_df[:train_size]

print ('Train shape: ',train_df_fit.shape)
print ('Test shape: ',test.shape)
print ('Y_df: ', Y_df_fit.shape)

Train shape:  (100000, 11)
Test shape:  (508438, 15)
Y_df:  (100000,)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Iniciaremos com o método Árvore de Decisão

In [16]:
#fazendo o treinamento com fit
clf_tree = tree.DecisionTreeClassifier()
clf_tree = clf_tree.fit(train_df_fit, Y_df_fit)

train_df_test = train_df[100000:105000]
train_df_test['description'] = sparse_desc[100000:105000]
Y_df_test = Y_df[100000:105000]
clf_tree.score(train_df_test, Y_df_test)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0.48859999999999998

SVM

In [ ]:
#fazendo o treinamento com fit
clf_svc = svm.SVC()
clf_svc = clf_svc.fit(train_df_fit, Y_df_fit)
clf_svc.score(train_df_test, Y_df_test)


/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:194: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


## Resultados
Como podemos ver acima, o método **COLOCAR MÉTODO AQUI** obteve melhor acurácia média - quanto o método acertou - dentre os 3 métodos testados. Por conta disto, ele será utilizado para prever as probabilidades correspondentes às informações contidas no *test*.

Mas, antes, os dados contidos em *test* também passarão pelo LabelEncoder, para codificação. Após, será utilizado o predict do método **COLOCAR NOME DO MÉTODO QUE SERÁ UTILIZADO**. Depois do predict, é utilizado o método inverse_transform da classe LabelEncoder para decodificar o resultado e saber os reais valores.

In [ ]:
#tirando colunas que não foram utilizadas
test_size = test.shape[0]

desc_test = test['description'][:test_size]
test_df = test.drop(['item_id','activation_date','user_type', 'title','description'], axis = 1)
test_df = test_df[:test_size]
print('New test: ', test_df.shape)

In [ ]:
#fazendo encoder
for col in test_df.columns:
    print (col)
    test_df[col] = enc[col].transform(test_df[col].values.astype(str))

desc_test_vec = hash_vect.transform(desc_test.values.astype(str))
test_df['description'] = pd.SparseArray(desc_test_vec.toarray().ravel(), fill_value = 0)[:test_size]

In [ ]:
#MUDAR CADA CLF PARA UM COM NOME DO MÉTODO CLF_MÉTODO
res = clf_svc.predict(test_df)
y_enc.inverse_transform(res)